In [1]:
import h5py
import sys
import os
sys.path.append("/home/jovyan/camels/proj1/")

from setup_params import get_photometry, get_luminosity_function, get_colour_distribution, get_available_snapshots#, get_colour_dir_name, get_magnitude_mask # get_safe_name
from variables_config import get_config


/home/jovyan/camels/proj1/1P_set/../camels.py:8: FutureWarning: The filters module has been moved to the instruments module. Please update your imports synthesizer.filters -> synthesizer.instruments
  from synthesizer.filters import Filter, FilterCollection


In [12]:
get_available_snapshots(photo_dir="/home/jovyan/Data/Photometry/Swift-EAGLE/L25n256/LH", model='Swift-EAGLE', sim_name='LH_0')



[]

In [ ]:
def explore_swift_eagle_files(input_dir):
    """
    Explore the structure of Swift-EAGLE HDF5 files to identify available data.
    
    Args:
        input_dir: Directory containing the HDF5 files
    """
    # Find all Swift-EAGLE photometry files
    photo_files = [f for f in os.listdir(input_dir) if f.endswith('_photometry.hdf5') and 'Swift-EAGLE' in f]
    
    if not photo_files:
        print(f"No Swift-EAGLE photometry files found in {input_dir}")
        return
    
    print(f"Found {len(photo_files)} Swift-EAGLE photometry files")
    
    for filename in photo_files:
        file_path = os.path.join(input_dir, filename)
        print(f"\nExploring {filename}:")
        
        try:
            with h5py.File(file_path, 'r') as f:
                # Get top-level keys (snapshots)
                top_keys = list(f.keys())
                print(f"Available snapshots: {top_keys}")
                
                # For each snapshot, explore further
                for snap_key in top_keys:
                    print(f"\n  Snapshot: {snap_key}")
                    
                    # Check for BC03 or other SPS models
                    snap_keys = list(f[snap_key].keys())
                    print(f"  Available groups: {snap_keys}")
                    
                    # If BC03 exists, check its structure
                    if "BC03" in snap_keys:
                        bc03_keys = list(f[f"{snap_key}/BC03"].keys())
                        print(f"  BC03 contains: {bc03_keys}")
                        
                        # Check photometry structure if it exists
                        if "photometry" in bc03_keys:
                            photo_keys = list(f[f"{snap_key}/BC03/photometry"].keys())
                            print(f"  Photometry contains: {photo_keys}")
                            
                            # Check for luminosity
                            if "luminosity" in photo_keys:
                                lum_keys = list(f[f"{snap_key}/BC03/photometry/luminosity"].keys())
                                print(f"  Luminosity contains: {lum_keys}")
                                
                                # Check for attenuated/intrinsic
                                for spec_type in ["attenuated", "intrinsic"]:
                                    if spec_type in lum_keys:
                                        spec_path = f"{snap_key}/BC03/photometry/luminosity/{spec_type}"
                                        filters = list(f[spec_path].keys())
                                        print(f"  {spec_type.capitalize()} filters: {filters}")
        
        except Exception as e:
            print(f"Error exploring {filename}: {str(e)}")
            import traceback
            traceback.print_exc()

if __name__ == "__main__":
    simulations = ["Swift-EAGLE"] # "SIMBA", "IllustrisTNG", "Astrid", 
    datasets = ["1P"]#"CV"]#,
    
    for simulation in simulations:
        for dataset in datasets:
            config = get_config(dataset=dataset, simulation=simulation)
            print(f"\nProcessing {dataset} {simulation}")

            input_dir = config["input_dir"]
            print("dir: ", input_dir)
            explore_swift_eagle_files(input_dir)

